In [1]:
import pandas as pd
df = pd.read_pickle("OPIF_04-06.pkl")

In [78]:
df[df.candidate == 'BUTTIGIEG']['callsign'].value_counts()[:20]

WMUR-TV    30
WBZ-TV     23
WCVB-TV    21
WPTZ       21
WFFF-TV    17
WFXT       16
KTTC       12
KSAX       12
WHDH       12
KRWF       12
KIMT       12
KBJR-TV    10
KRII       10
KFTC       10
WCAX-TV    10
KAAL        8
WAVY-TV     8
WHNS        7
WABM        7
KCNC-TV     6
Name: callsign, dtype: int64

In [79]:
df[df.candidate == 'KLOBUCHAR']['callsign'].value_counts()[:20]

KTTC       52
WMUR-TV    35
KXLT-TV    27
KIMT       17
KAAL       17
KVLY-TV    17
WDAY-TV    16
WDAZ-TV    12
KBJR-TV    11
WSOC-TV    11
KRII       11
WTVD       10
WHNS        9
KOCO-TV     9
WVBT        9
KFTC        8
WSMV-TV     8
WAXN-TV     8
KMSP-TV     8
WAVY-TV     8
Name: callsign, dtype: int64

In [19]:
dfa = df[df.candidate == 'KLOBUCHAR'].groupby('callsign').first()

In [20]:
dfb = df[df.candidate == 'BUTTIGIEG'].groupby('callsign').first()

In [24]:
import os
try:
    os.mkdir('Buttigieg Files/')
    os.mkdir('Klobuchar Files/')
except OSError:
    print ("Creation of the directory failed")
else:
    print ("Successfully created the directory")

Successfully created the directory


In [25]:
import requests, requests_cache, time
from IPython.display import clear_output
requests_cache.install_cache()

In [30]:
dfa.reset_index(inplace = True)
dfb.reset_index(inplace = True)

In [33]:
n = 0
errorCount = 0
for url, name in zip(dfa.file_url, dfa.file_name):
    print("Downloading file {}/{}".format(n+1,len(dfa)))
    
    r = requests.get(url, stream = True)
    if (r.status_code != 200): 
        print(url, r.text)
        errorCount += 1
        time.sleep(2)
    else: 
        with open('Klobuchar Files/' + name + '.pdf', 'wb') as f:
            f.write(r.content)
        dfa.loc[n,'file_path'] = 'Klobuchar Files/' + name + '.pdf'
    if not getattr(r, 'from_cache', False):
        time.sleep(0.25)
    clear_output(wait = True)
    n += 1
print("Number of errors: ", errorCount)

Number of errors:  1


In [35]:
n = 0
errorCount = 0
for url, name in zip(dfb.file_url, dfb.file_name):
    print("Downloading file {}/{}".format(n+1,len(dfb)))
    
    r = requests.get(url, stream = True)
    if (r.status_code != 200): 
        print(url, r.text)
        errorCount += 1
        time.sleep(2)
    else: 
        with open('Buttigieg Files/' + name + '.pdf', 'wb') as f:
            f.write(r.content)
        dfb.loc[n,'file_path'] = 'Buttigieg Files/' + name + '.pdf'
    if not getattr(r, 'from_cache', False):
        time.sleep(0.25)
    clear_output(wait = True)
    n += 1
print("Number of errors: ", errorCount)

Number of errors:  3


In [62]:
## count of 6 for WTVF
dfa[dfa.file_path.isnull()]

,callsign,candidate,entity_id,nielsen_dma_rank,create_ts,file_url,file_name,file_status,file_id,network_affiliation,active_ind,date,file_path
92,WTVF,KLOBUCHAR,36504,NASHVILLE,2020-02-20 19:27:18.857000+00:00,https://publicfiles.fcc.gov/api/manager/downlo...,Amy Klobuchar Disclosure Form WTVF 2-19,com_prc,9432083d-64ed-daad-37d8-54c3a328c573,CBS,Y,2020-02-20,NaN


In [85]:
pd.isna(dfa[dfa.callsign == 'WTVF']['file_path'])

92    True
Name: file_path, dtype: bool

In [64]:
## counts of 10 --> 4 --> 3 files for KFTC, KMGH-TV, and WLOS
dfb[dfb.file_path.isnull()]

,callsign,candidate,entity_id,nielsen_dma_rank,create_ts,file_url,file_name,file_status,file_id,network_affiliation,active_ind,date,file_path
11,KFTC,BUTTIGIEG,83714,MINNEAPOLIS-ST. PAUL,2020-02-25 20:11:21.620000+00:00,https://publicfiles.fcc.gov/api/manager/downlo...,Pete Buttigieg-Disclosure form,com_prc,e3f079a9-d5e5-6518-ef0d-6014e75a42b3,FOX,Y,2020-02-25,NaN
14,KMGH-TV,BUTTIGIEG,40875,DENVER,2020-02-25 20:15:17.974000+00:00,https://publicfiles.fcc.gov/api/manager/downlo...,Buttigieg Disclosure 2.25,com_prc,22df6a36-1d4c-65e1-6585-e6c1527666e0,ABC,N,2020-02-25,NaN
63,WLOS,BUTTIGIEG,56537,GREENVLL-SPART-ASHEVLL-AND,2020-03-10 18:55:43.307000+00:00,https://publicfiles.fcc.gov/api/manager/downlo...,Pete Buttigieg Rebates week of 2.23,com_prc,ea18943d-ff39-b678-7cc5-b40fbedd43c1,ABC,Y,2020-03-10,NaN


In [107]:
dfa.drop(columns = {'create_ts'}).to_excel("Klobuchar_stations.xlsx")
dfb.drop(columns = {'create_ts'}).to_excel("Buttigieg_stations.xlsx")

In [116]:
dfb_x = df[(df.candidate == 'BUTTIGIEG') & ((df.callsign == 'KFTC') | (df.callsign == 'KMGH-TV') | (df.callsign == 'WLOS'))]

In [117]:
dfa_x =  df[(df.candidate == 'KLOBUCHAR') & (df.callsign == 'WTVF')]

In [118]:
dfa_x.drop(columns = {'create_ts'}).to_excel("Klob_extra.xlsx")
dfb_x.drop(columns = {'create_ts'}).to_excel("Butt_extra.xlsx")